<a href="https://colab.research.google.com/github/GoldenWanGila/Collage-Project/blob/master/DEF%E8%87%AA%E5%BC%B7%E4%B8%AD%E9%83%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
from re import T
import sklearn.linear_model
import seaborn as sns; sns.set()
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sympy import *
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import matplotlib as mpl
import pandas as pd
import numpy as np
import datetime

In [69]:
def plot(x:np.ndarray, y:np.ndarray):
    x = x.reshape(-1,1)
    xgbrModel=xgb.XGBRegressor()
    xgbrModel.fit(x,y)
    predicted=xgbrModel.predict(x)
    v = mean_squared_error(y,predicted)
    print(sqrt(v))

In [70]:
def arrangeDataFrame(x_dataFrame:pd.DataFrame,y_dataFrame:pd.DataFrame,direction:int):
    x_dataFrame_direction = x_dataFrame[x_dataFrame['direction']==direction]
    y_dataFrame_direction = y_dataFrame[y_dataFrame['direction']==direction]

    if len(set(x_dataFrame_direction['date'])) != len(set(y_dataFrame_direction['date'])):
        same_dates = [date for date in set(y_dataFrame_direction['date']) if (date in set(x_dataFrame_direction['date']))]
        x_dataFrame_direction = x_dataFrame_direction[x_dataFrame_direction['date'].isin(same_dates)]
        y_dataFrame_direction = y_dataFrame_direction[y_dataFrame_direction['date'].isin(same_dates)]

    same_nums = [num for num in set(y_dataFrame_direction['num']) if (num in set(x_dataFrame_direction['num']))]
    x_target_dataFrame = x_dataFrame_direction[x_dataFrame_direction['num'].isin(same_nums)]
    x_target_dataFrame.sort_values(by=['date','num'], inplace=True)
    x_target_dataFrame.reset_index(inplace=True)
    y_target_dataFrame = y_dataFrame_direction[y_dataFrame_direction['num'].isin(same_nums)]
    y_target_dataFrame.sort_values(by=['date','num'], inplace=True)
    y_target_dataFrame.reset_index(inplace=True)

    # 獲取 delay不存在的 row index
    null_index = list(x_target_dataFrame[x_target_dataFrame['delay'].isnull()].index)
    null_index.extend(list(y_target_dataFrame[y_target_dataFrame['delay'].isnull()].index))
    null_index = set(null_index)

    # 將delay不存在的 row 從資料中移除
    x_target_dataFrame = x_target_dataFrame.drop(index=null_index)
    y_target_dataFrame = y_target_dataFrame.drop(index=null_index)

    x = x_target_dataFrame['delay'].to_numpy()
    y = y_target_dataFrame['delay'].to_numpy()
    return x,y

In [71]:
def getRMSE(x:np.ndarray, y:np.ndarray):
    x = x.reshape(-1,1)
    xgbrModel=xgb.XGBRegressor()
    xgbrModel.fit(x,y)
    predicted=xgbrModel.predict(x)
    v = mean_squared_error(y,predicted)
    return sqrt(v)

In [72]:
dataFrame = pd.read_csv('/content/drive/MyDrive/railway.csv')
label_map = {'Keelung':1,'Xizhi':2,'Taipei':3,'Banqiao':4,'Taoyuan':5,'Zhongli':6,'Hsinchu':7,'Zhunan':8,'Miaoli':9,'Fengyuan':10,'Taichung':11,'Changhua':12,'Yuanlin':13,
'Douliu':14,'Chiayi':15,'Tainan':16,'Xinzuoying':17,'Kaohsiung':18,'Tanzi':0,'Taiyuan':0,'Xinwuri':0}

In [73]:
dataFrame_localTrain = dataFrame[dataFrame['num']<500]
dataFrame_localTrain.sort_values(by=["time","date"],inplace=True)
dataFrame_localTrain = dataFrame_localTrain[dataFrame_localTrain["date"]>='2021/01/22']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


**連假日**

In [74]:
listlist = ["2021/02/10","2021/02/11","2021/02/12","2021/02/13","2021/02/14","2021/02/15","2021/02/16","2021/04/02","2021/04/03","2021/04/04","2021/04/05","2021/02/26","2021/02/27","2021/02/28","2021/03/01"]

In [75]:
dataFrame_holiday = dataFrame_localTrain[dataFrame_localTrain["date"].isin(listlist)]

In [76]:

dataFrame_holiday_Hsinchu = dataFrame_holiday[dataFrame_holiday['Station']=='Hsinchu']
dataFrame_holiday_Zhunan = dataFrame_holiday[dataFrame_holiday['Station']=='Zhunan']
dataFrame_holiday_Miaoli = dataFrame_holiday[dataFrame_holiday['Station']=='Miaoli']
dataFrame_holiday_Fengyuan = dataFrame_holiday[dataFrame_holiday['Station']=='Fengyuan']
dataFrame_holiday_Taichung = dataFrame_holiday[dataFrame_holiday['Station']=="Taichung"]
dataFrame_holiday_Changhua = dataFrame_holiday[dataFrame_holiday['Station']=='Changhua']
dataFrame_holiday_Yuanlin = dataFrame_holiday[dataFrame_holiday['Station']=='Yuanlin']
dataFrame_holiday_Douliu = dataFrame_holiday[dataFrame_holiday['Station']=='Douliu']

In [77]:
dataFrameList = [dataFrame_holiday_Hsinchu, dataFrame_holiday_Zhunan, dataFrame_holiday_Miaoli, dataFrame_holiday_Fengyuan, dataFrame_holiday_Taichung, dataFrame_holiday_Changhua, 
                 dataFrame_holiday_Yuanlin,dataFrame_holiday_Douliu]
mseList = []
for positionStart in range(len(dataFrameList)):
    mseList.append([])
    dataFrameStart = dataFrameList[positionStart]
    for positionEnd in range(len(dataFrameList)):
        dataFrameEnd = dataFrameList[positionEnd]
        if positionStart > positionEnd:  # 順行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 0)
            mseList[positionStart].append(getRMSE(x,y))
        elif positionStart < positionEnd: # 逆行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 1)
            mseList[positionStart].append(getRMSE(y,x))
        else:
            mseList[positionStart].append('--')
stationList = ['Hsinchu','Zhunan','Miaoli','Fengyuani','Taichung','Changhua','Yuanlin','Douliu']
mseDict = dict(zip(stationList, mseList))
mse_df = pd.DataFrame(mseDict, index=stationList)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.

[12:35:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

[12:35:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.

[12:35:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

[12:35:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/l

In [78]:
mse_df.to_csv("中部連假日MSE.csv")
mse_df

,Hsinchu,Zhunan,Miaoli,Fengyuani,Taichung,Changhua,Yuanlin,Douliu
Hsinchu,--,0.978349793353832,2.54013267886793,2.53609554501613,2.35271618707377,3.78821595572234,4.39345340912768,2.52472869538751
Zhunan,1.21501727630591,--,1.04922552124455,1.73820164080619,1.82855265127864,4.18831540640663,4.36117990340493,2.54131781888880
Miaoli,8.97324786394316,0.591978774461224,--,1.14156074889652,1.26233734811533,2.02867027209988,2.19015590235396,1.97938160885828
Fengyuani,1.55435723082573,1.04180859362373,0.882144386224419,--,1.02183722143771,1.46096086315164,1.75250129476430,1.81631572023638
Taichung,1.72595459754295,1.39820236600923,1.18299724862439,0.904954889166207,--,1.07239469985279,1.39164894935056,1.62796880105757
Changhua,2.34287930972306,2.06828355158293,1.42538577208833,1.37719045378672,1.07402604019298,--,0.853565332161804,1.44998199874444
Yuanlin,2.53703949213460,2.19831127096734,1.56972049671485,1.55088472571776,1.23332004255336,0.904443105968481,--,1.18376354473664
Douliu,2.36524842667345,2.28536172618261,1.76005356170701,1.76234250782725,1.56145958277216,1.27490229254549,0.996035906849632,--


**全部**

In [79]:
dataFrame_localTrain = dataFrame_localTrain[dataFrame_localTrain["date"]!="2021/04/02"]
dataFrame_localTrain = dataFrame_localTrain[dataFrame_localTrain["date"]!="2021/04/03"]

In [80]:
dataFrame_localTrain_Hsinchu = dataFrame_localTrain[dataFrame_localTrain['Station']=='Hsinchu']
dataFrame_localTrain_Zhunan = dataFrame_localTrain[dataFrame_localTrain['Station']=='Zhunan']
dataFrame_localTrain_Miaoli = dataFrame_localTrain[dataFrame_localTrain['Station']=='Miaoli']
dataFrame_localTrain_Fengyuan = dataFrame_localTrain[dataFrame_localTrain['Station']=='Fengyuan']
dataFrame_localTrain_Taichung = dataFrame_localTrain[dataFrame_localTrain['Station']=="Taichung"]
dataFrame_localTrain_Changhua = dataFrame_localTrain[dataFrame_localTrain['Station']=='Changhua']
dataFrame_localTrain_Yuanlin = dataFrame_localTrain[dataFrame_localTrain['Station']=='Yuanlin']
dataFrame_localTrain_Douliu = dataFrame_localTrain[dataFrame_localTrain['Station']=='Douliu']

In [81]:
dataFrameList = [dataFrame_localTrain_Hsinchu, dataFrame_localTrain_Zhunan, dataFrame_localTrain_Miaoli, dataFrame_localTrain_Fengyuan, dataFrame_localTrain_Taichung, dataFrame_localTrain_Changhua, 
                 dataFrame_localTrain_Yuanlin,dataFrame_localTrain_Douliu]
mseList = []
for positionStart in range(len(dataFrameList)):
    mseList.append([])
    dataFrameStart = dataFrameList[positionStart]
    for positionEnd in range(len(dataFrameList)):
        dataFrameEnd = dataFrameList[positionEnd]
        if positionStart > positionEnd:  # 順行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 0)
            mseList[positionStart].append(getRMSE(x,y))
        elif positionStart < positionEnd: # 逆行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 1)
            mseList[positionStart].append(getRMSE(y,x))
        else:
            mseList[positionStart].append('--')
stationList = ['Hsinchu','Zhunan','Miaoli','Fengyuani','Taichung','Changhua','Yuanlin','Douliu']
mseDict = dict(zip(stationList, mseList))
mse_df = pd.DataFrame(mseDict, index=stationList)
mse_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.

[12:35:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

[12:35:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

...

[12:35:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

[12:35:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/l

,Hsinchu,Zhunan,Miaoli,Fengyuani,Taichung,Changhua,Yuanlin,Douliu
Hsinchu,--,0.985712837705084,1.83206359974947,2.07418405173839,2.08837687880475,2.35544648584548,2.48294661723655,2.43522135033566
Zhunan,0.978714801086018,--,1.10511495037548,1.71495862919681,1.85830772991433,2.33268936368783,2.41649021194118,2.48632394211464
Miaoli,1.19944279299259,0.649325230367047,--,1.48327561423755,1.61267561811399,1.85985901541113,2.02386264447552,2.16470820549889
Fengyuani,1.95634288030337,1.78761694483466,1.67379713843141,--,0.863442160351943,1.30996750987228,1.71442048003737,1.94730933918380
Taichung,2.06933359662115,1.96801187518843,1.83189592163742,0.932016830051303,--,1.00341435449175,1.47706836191421,1.72082876772287
Changhua,2.80124294562485,2.86238033394229,2.05876227708942,1.30122313823278,1.02191160927636,--,1.03353399489176,1.52300534279945
Yuanlin,3.01579540633774,2.99726921539268,2.16811607754803,1.47457451397143,1.23985138306045,0.857529282218033,--,1.22413899385033
Douliu,3.08492060434897,3.16472449289108,2.27462570940382,1.69784460963906,1.51918063400115,1.22939047018024,1.07063877574045,--


In [82]:
dataFrame_localTrain_weekday = dataFrame_localTrain[dataFrame_localTrain['date'].apply(lambda date: datetime.date(int(date[:4]), int(date[5:7]), int(date[8:])).isoweekday() < 6)]
dataFrame_localTrain_weekday

,Station,direction,id,date,time,num,dest,delay,type
242276,Taipei,0,1,2021/01/22,00:02,251,Shulin,4.0,B
242748,Taipei,0,1,2021/01/25,00:02,251,Shulin,3.0,B
242904,Taipei,0,1,2021/01/26,00:02,251,Shulin,0.0,B
243056,Taipei,0,1,2021/01/27,00:02,251,Shulin,1.0,B
243208,Taipei,0,1,2021/01/28,00:02,251,Shulin,0.0,B
...,...,...,...,...,...,...,...,...,...
312912,Kaohsiung,0,96,2021/03/30,23:55,149,Chaozhou,16.0,B
313007,Kaohsiung,0,95,2021/03/31,23:55,149,Chaozhou,0.0,B
313102,Kaohsiung,0,95,2021/04/01,23:55,149,Chaozhou,0.0,B
313486,Kaohsiung,0,95,2021/04/05,23:55,149,Chaozhou,21.0,B


In [83]:
dataFrame_localTrain_weekend = dataFrame_localTrain[dataFrame_localTrain['date'].apply(lambda date: datetime.date(int(date[:4]), int(date[5:7]), int(date[8:])).isoweekday() > 5)]
dataFrame_localTrain_weekend

,Station,direction,id,date,time,num,dest,delay,type
242433,Taipei,0,1,2021/01/23,00:02,251,Shulin,5.0,B
242587,Taipei,0,1,2021/01/24,00:02,249,Shulin,3.0,B
243376,Taipei,0,1,2021/01/30,00:02,251,Shulin,0.0,B
243515,Taipei,0,1,2021/01/31,00:02,249,Shulin,6.0,B
244585,Taipei,0,1,2021/02/06,00:02,251,Shulin,0.0,B
...,...,...,...,...,...,...,...,...,...
311954,Kaohsiung,0,95,2021/03/20,23:55,149,Chaozhou,2.0,B
312051,Kaohsiung,0,97,2021/03/21,23:55,149,Chaozhou,0.0,B
312623,Kaohsiung,0,95,2021/03/27,23:55,149,Chaozhou,0.0,B
312720,Kaohsiung,0,97,2021/03/28,23:55,149,Chaozhou,3.0,B


**假日**

In [84]:
dataFrame_localTrain_weekend_Hsinchu = dataFrame_localTrain_weekend[dataFrame_localTrain_weekend['Station']=='Hsinchu']
dataFrame_localTrain_weekend_Zhunan = dataFrame_localTrain_weekend[dataFrame_localTrain_weekend['Station']=='Zhunan']
dataFrame_localTrain_weekend_Miaoli = dataFrame_localTrain_weekend[dataFrame_localTrain_weekend['Station']=='Miaoli']
dataFrame_localTrain_weekend_Fengyuan = dataFrame_localTrain_weekend[dataFrame_localTrain_weekend['Station']=='Fengyuan']
dataFrame_localTrain_weekend_Taichung = dataFrame_localTrain_weekend[dataFrame_localTrain_weekend['Station']=="Taichung"]
dataFrame_localTrain_weekend_Changhua = dataFrame_localTrain_weekend[dataFrame_localTrain_weekend['Station']=='Changhua']
dataFrame_localTrain_weekend_Yuanlin = dataFrame_localTrain_weekend[dataFrame_localTrain_weekend['Station']=='Yuanlin']
dataFrame_localTrain_weekend_Douliu = dataFrame_localTrain_weekend[dataFrame_localTrain_weekend['Station']=='Douliu']

In [85]:
dataFrameList = [dataFrame_localTrain_weekend_Hsinchu,dataFrame_localTrain_weekend_Zhunan,dataFrame_localTrain_weekend_Miaoli,dataFrame_localTrain_weekend_Fengyuan,dataFrame_localTrain_weekend_Taichung,dataFrame_localTrain_weekend_Changhua,
                 dataFrame_localTrain_weekend_Yuanlin,dataFrame_localTrain_weekend_Douliu]
mseList = []
for positionStart in range(len(dataFrameList)):
    mseList.append([])
    dataFrameStart = dataFrameList[positionStart]
    for positionEnd in range(len(dataFrameList)):
        dataFrameEnd = dataFrameList[positionEnd]
        if positionStart > positionEnd:  # 順行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 0)
            mseList[positionStart].append(getRMSE(x,y))
        elif positionStart < positionEnd: # 逆行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 1)
            mseList[positionStart].append(getRMSE(y,x))
        else:
            mseList[positionStart].append('--')

stationList = ['Hsinchu','Zhunan','Miaoli','Fengyuani','Taichung','Changhua','Yuanlin','Douliu']
mseDict = dict(zip(stationList, mseList))
mse_df = pd.DataFrame(mseDict, index=stationList)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.

[12:35:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/l

[12:35:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.

[12:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

[12:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

In [86]:
mse_df.to_csv("中部假日MSE.csv")
mse_df

,Hsinchu,Zhunan,Miaoli,Fengyuani,Taichung,Changhua,Yuanlin,Douliu
Hsinchu,--,1.01356259068438,1.92451366748733,1.98858543002918,2.11973091597615,2.17441405286430,2.34907516294195,2.09636609241568
Zhunan,0.809338379693436,--,1.02446441122942,1.77207173598942,1.83118790752539,2.10407097302267,2.10872382108692,2.10301256252516
Miaoli,1.17927736225353,0.680371117188876,--,1.53734989896914,1.71045465598718,1.77709999434797,1.72197458921771,1.77397889553161
Fengyuani,2.70265798503284,2.57489279799535,2.46249412434742,--,0.915088203513870,1.24563087803174,1.43154247491802,1.66153681422369
Taichung,2.74268276085179,2.63737032035574,2.49557243090969,0.896335321457454,--,0.979091097994795,1.24058279139674,1.49890528548391
Changhua,4.20443842579099,4.43086279032662,2.84350738840044,1.35885707914831,1.14582554677509,--,0.691786295532000,1.28300007906644
Yuanlin,4.45050345694391,4.52266518197310,2.94825105917374,1.54502257930982,1.30350209817521,0.833991229149296,--,1.13941623987025
Douliu,4.57289944536710,4.80715014383871,3.06204882395374,1.78988421655569,1.65261393814660,1.24785573055632,1.09492166190455,--


**平日**

In [87]:
dataFrame_localTrain_weekday_Hsinchu = dataFrame_localTrain_weekday[dataFrame_localTrain_weekday['Station']=='Hsinchu']
dataFrame_localTrain_weekday_Zhunan = dataFrame_localTrain_weekday[dataFrame_localTrain_weekday['Station']=='Zhunan']
dataFrame_localTrain_weekday_Miaoli = dataFrame_localTrain_weekday[dataFrame_localTrain_weekday['Station']=='Miaoli']
dataFrame_localTrain_weekday_Fengyuan = dataFrame_localTrain_weekday[dataFrame_localTrain_weekday['Station']=='Fengyuan']
dataFrame_localTrain_weekday_Taichung = dataFrame_localTrain_weekday[dataFrame_localTrain_weekday['Station']=="Taichung"]
dataFrame_localTrain_weekday_Changhua = dataFrame_localTrain_weekday[dataFrame_localTrain_weekday['Station']=='Changhua']
dataFrame_localTrain_weekday_Yuanlin = dataFrame_localTrain_weekday[dataFrame_localTrain_weekday['Station']=='Yuanlin']
dataFrame_localTrain_weekday_Douliu = dataFrame_localTrain_weekday[dataFrame_localTrain_weekday['Station']=='Douliu']

In [88]:
dataFrameList = [dataFrame_localTrain_weekday_Hsinchu,dataFrame_localTrain_weekday_Zhunan,dataFrame_localTrain_weekday_Miaoli,dataFrame_localTrain_weekday_Fengyuan,dataFrame_localTrain_weekday_Taichung,dataFrame_localTrain_weekday_Changhua,
                 dataFrame_localTrain_weekday_Yuanlin,dataFrame_localTrain_weekday_Douliu]
mseList = []
for positionStart in range(len(dataFrameList)):
    mseList.append([])
    dataFrameStart = dataFrameList[positionStart]
    for positionEnd in range(len(dataFrameList)):
        dataFrameEnd = dataFrameList[positionEnd]
        if positionStart > positionEnd:  # 順行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 0)
            mseList[positionStart].append(getRMSE(x,y))
        elif positionStart < positionEnd: # 逆行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 1)
            mseList[positionStart].append(getRMSE(y,x))
        else:
            mseList[positionStart].append('--')

stationList = ['Hsinchu','Zhunan','Miaoli','Fengyuani','Taichung','Changhua','Yuanlin','Douliu']
mseDict = dict(zip(stationList, mseList))
mse_df = pd.DataFrame(mseDict, index=stationList)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.

[12:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/l

[12:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

[12:35:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

[12:35:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:35:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [89]:
mse_df.to_csv("中部平日MSE.csv")
mse_df

,Hsinchu,Zhunan,Miaoli,Fengyuani,Taichung,Changhua,Yuanlin,Douliu
Hsinchu,--,0.944957206348117,1.71568865740907,1.86852464913685,1.92205852667891,2.29534020524384,2.23663765771882,2.24487182645250
Zhunan,1.01747726637854,--,1.09105626403075,1.61556543231927,1.71399901245290,2.35588868628534,2.22768609770830,2.27271131665133
Miaoli,1.18978877544059,0.626662334154259,--,1.20534359346736,1.35386402754394,1.70614845998601,1.79554865384524,1.95355017667498
Fengyuani,1.48847299634879,1.29386774287979,1.16559298346599,--,0.771928348936287,1.26315884203703,1.50815982375597,1.79217563841493
Taichung,1.63698104758056,1.53935562517939,1.42900996891250,0.929698425168404,--,0.979272615978290,1.24248368863747,1.57332621109798
Changhua,1.87753192126057,1.79950159043698,1.57317770209653,1.25866598671605,0.946945758721596,--,0.783582800425280,1.39563903375047
Yuanlin,2.00531488904640,1.92103115471917,1.68245521350308,1.40563410626346,1.17437283952982,0.857453368679126,--,1.24064880359830
Douliu,2.08238557771381,2.05073899541267,1.79744651905890,1.62557746675445,1.41592849050631,1.20893936174121,1.04997536127267,--
